In [12]:
import gymnasium as gym
from imitation.algorithms.adversarial.gail import GAIL
from imitation.data.wrappers import RolloutInfoWrapper
from imitation.rewards.reward_nets import BasicRewardNet
from imitation.util.networks import RunningNorm
from imitation.rewards.reward_nets import BasicRewardNet
from stable_baselines3 import PPO
from stable_baselines3.a2c import MlpPolicy
from imitation.data import rollout
import numpy as np
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common import monitor
from Wordle import WordleEnv
import os

env = monitor.Monitor(WordleEnv())
venv = DummyVecEnv([lambda: RolloutInfoWrapper(env)])  # Wrap a single environment -- only useful for simple testing like this
venv.render_mode="human"

#rollouts = trajectories
rollouts = np.load('data/trajectories_all.npy', allow_pickle=True)

transitions = rollout.flatten_trajectories_with_rew(rollouts)

In [13]:
#discriminator
learner = PPO(
    env=env,
    policy=MlpPolicy,
    batch_size=64,
    ent_coef=0.0,
    learning_rate=0.0004,
    gamma=0.95,
    n_epochs=5,
    seed=None,
)

#generator
reward_net = BasicRewardNet(
    observation_space=env.observation_space,
    action_space=env.action_space,
    normalize_input_layer=RunningNorm,
)

#GAIL
gail_trainer = GAIL(
    demonstrations=transitions,
    demo_batch_size=1024,
    gen_replay_buffer_capacity=512,
    n_disc_updates_per_round=8,
    venv=venv,
    gen_algo=learner,
    reward_net=reward_net,
    allow_variable_horizon=True
)

Running with `allow_variable_horizon` set to True. Some algorithms are biased towards shorter or longer episodes, which may significantly confound results. Additionally, even unbiased algorithms can exploit the information leak from the termination condition, producing spuriously high performance. See https://imitation.readthedocs.io/en/latest/getting-started/variable-horizon.html for more information.


In [16]:
# evaluate the learner before training
learner_rewards_before_training, _ = evaluate_policy(
    learner, venv, 100, return_episode_rewards=True,
)

In [20]:
# train the learner and evaluate again
gail_trainer.train(100000)  # Train for 800_000 steps to match expert.
learner_rewards_after_training, _ = evaluate_policy(
    learner, venv, 100, return_episode_rewards=True,
)

round:   0%|          | 0/48 [00:00<?, ?it/s]

------------------------------------------
| raw/                        |          |
|    gen/rollout/ep_len_mean  | 6        |
|    gen/rollout/ep_rew_mean  | -8.25    |
|    gen/time/fps             | 72       |
|    gen/time/iterations      | 1        |
|    gen/time/time_elapsed    | 28       |
|    gen/time/total_timesteps | 2048     |
------------------------------------------
--------------------------------------------------
| raw/                                |          |
|    disc/disc_acc                    | 0.499    |
|    disc/disc_acc_expert             | 0.998    |
|    disc/disc_acc_gen                | 0        |
|    disc/disc_entropy                | 0.692    |
|    disc/disc_loss                   | 0.695    |
|    disc/disc_proportion_expert_pred | 0.999    |
|    disc/disc_proportion_expert_true | 0.5      |
|    disc/global_step                 | 1        |
|    disc/n_expert                    | 1.02e+03 |
|    disc/n_generated                 | 1.02e+03 |
-

round:   2%|▏         | 1/48 [00:37<29:16, 37.37s/it]

---------------------------------------------------
| raw/                               |            |
|    gen/rollout/ep_len_mean         | 6          |
|    gen/rollout/ep_rew_mean         | -8.12      |
|    gen/rollout/ep_rew_wrapped_mean | 4.38       |
|    gen/time/fps                    | 71         |
|    gen/time/iterations             | 1          |
|    gen/time/time_elapsed           | 28         |
|    gen/time/total_timesteps        | 4096       |
|    gen/train/approx_kl             | 0.04380278 |
|    gen/train/clip_fraction         | 0.492      |
|    gen/train/clip_range            | 0.2        |
|    gen/train/entropy_loss          | -9.47      |
|    gen/train/explained_variance    | 0.0228     |
|    gen/train/learning_rate         | 0.0004     |
|    gen/train/loss                  | -0.131     |
|    gen/train/n_updates             | 5          |
|    gen/train/policy_gradient_loss  | -0.113     |
|    gen/train/value_loss            | 0.383      |
------------

round:   4%|▍         | 2/48 [01:15<28:46, 37.53s/it]

...

round:  96%|█████████▌| 46/48 [31:10<01:24, 42.27s/it]

----------------------------------------------------
| raw/                               |             |
|    gen/rollout/ep_len_mean         | 5.96        |
|    gen/rollout/ep_rew_mean         | -7.07       |
|    gen/rollout/ep_rew_wrapped_mean | 1.83        |
|    gen/time/fps                    | 62          |
|    gen/time/iterations             | 1           |
|    gen/time/time_elapsed           | 32          |
|    gen/time/total_timesteps        | 96256       |
|    gen/train/approx_kl             | 0.042109817 |
|    gen/train/clip_fraction         | 0.182       |
|    gen/train/clip_range            | 0.2         |
|    gen/train/entropy_loss          | -2.89       |
|    gen/train/explained_variance    | 0.133       |
|    gen/train/learning_rate         | 0.0004      |
|    gen/train/loss                  | 0.626       |
|    gen/train/n_updates             | 230         |
|    gen/train/policy_gradient_loss  | -0.0322     |
|    gen/train/value_loss            | 0.977  

round:  98%|█████████▊| 47/48 [31:52<00:42, 42.17s/it]

----------------------------------------------------
| raw/                               |             |
|    gen/rollout/ep_len_mean         | 6           |
|    gen/rollout/ep_rew_mean         | -7.61       |
|    gen/rollout/ep_rew_wrapped_mean | 1.86        |
|    gen/time/fps                    | 65          |
|    gen/time/iterations             | 1           |
|    gen/time/time_elapsed           | 31          |
|    gen/time/total_timesteps        | 98304       |
|    gen/train/approx_kl             | 0.096188486 |
|    gen/train/clip_fraction         | 0.333       |
|    gen/train/clip_range            | 0.2         |
|    gen/train/entropy_loss          | -3.5        |
|    gen/train/explained_variance    | 0.165       |
|    gen/train/learning_rate         | 0.0004      |
|    gen/train/loss                  | 0.62        |
|    gen/train/n_updates             | 235         |
|    gen/train/policy_gradient_loss  | -0.0474     |
|    gen/train/value_loss            | 1.2    

round: 100%|██████████| 48/48 [32:32<00:00, 40.68s/it]


In [26]:
print(
    "Rewards before training:",
    np.mean(learner_rewards_before_training),
    "+/-",
    np.std(learner_rewards_before_training),
)
print(
    "Rewards after training:",
    np.mean(learner_rewards_after_training),
    "+/-",
    np.std(learner_rewards_after_training),
)

Rewards before training: -8.5 +/- 2.2561028345356955
Rewards after training: -6.0 +/- 0.0
